In [ ]:
import cv2
import numpy as np

from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [20, 10]

from ai_ct_scans import data_loading,

In [ ]:
# Load full scan data
patient_dir = data_loading.data_root_directory() / "1"
patient_loader = data_loading.PatientLoader(patient_dir)

patient_loader.abdo.scan_1.load_scan()
patient_loader.abdo.scan_2.load_scan()

scan_1_mid = 255
scan_2_mid = 255

# Plot some full scale data
f, axarr = plt.subplots(1, 2)
axarr[0].imshow(patient_loader.abdo.scan_1.full_scan[200, :, :])
axarr[0].title.set_text("First scan")
axarr[1].imshow(patient_loader.abdo.scan_2.full_scan[200, :, :])
axarr[1].title.set_text("Second scan")

In [ ]:
# Take slice through sagittal plane
sagittal_slice = patient_loader.abdo.scan_1.full_scan[:, :, 200]
plt.imshow(sagittal_slice)

In [ ]:
# Binarise slice through sagital plane
_, binary_slice = cv2.threshold(sagittal_slice, 500, 1, cv2.THRESH_BINARY)
plt.imshow(binary_slice)

In [ ]:
# Do edge detection on binarised image
edges = cv2.cv2.Sobel(binary_slice, cv2.CV_8U, 1, 0, ksize=3)
plt.imshow(edges)

In [ ]:
# Apply hough transform to find line that corresponds to table
lines = cv2.HoughLines(edges, 1, np.pi / 180, int(edges.shape[0] * 0.5), None, 0, 0)
table_locations = []
for line in lines:
    for i in line:
        table_locations.append(i[0])

print(table_locations)
print(lines[:, 0, 0])

In [ ]:
# Apply over full volume


def find_table(slice_image):
    _, binary_slice = cv2.threshold(slice_image, 800, 1, cv2.THRESH_BINARY)
    edges = cv2.cv2.Sobel(binary_slice, cv2.CV_8U, 1, 0, ksize=3)
    lines = cv2.HoughLines(edges, 1, np.pi / 180, int(edges.shape[0] * 0.5), None, 0, 0)
    table_locations = []
    if lines is None or lines.size == 0:
        return
    return int(min(lines[:, 0, 0]))


sagittal_mid = patient_loader.abdo.scan_1.full_scan.shape[2] // 2
sagittal_slice = patient_loader.abdo.scan_1.full_scan[:, :, sagittal_mid]
x = []
y = []

x.append(sagittal_mid)
y.append(find_table(sagittal_slice))

i = sagittal_mid
while i < patient_loader.abdo.scan_1.full_scan.shape[2] - 1:
    i += 1
    table = find_table(patient_loader.abdo.scan_1.full_scan[:, :, i])
    if table:
        x.append(i)
        y.append(table)
    else:
        break

found = True
i = sagittal_mid
while i > 1:
    i -= 1
    table = find_table(patient_loader.abdo.scan_1.full_scan[:, :, i])
    if table:
        x.append(i)
        y.append(table)
    else:
        break

plt.imshow(patient_loader.abdo.scan_1.full_scan[300, :, :])
plt.scatter(x, y, c="r")

In [ ]:
# Remove table from scan
scan = patient_loader.abdo.scan_1.full_scan

for i in range(len(x)):
    scan[:, y[i] - 20 :, x[i]] = 0

plt.imshow(scan[300, :, :])